<a href="https://colab.research.google.com/github/minsuk-heo/tf2/blob/master/jupyter_notebooks/10.Word2Vec_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd

import tensorflow_hub as hub
import numpy as np

In [3]:
# Load Pretrained Word2Vec
embed = hub.load("https://tfhub.dev/google/Wiki-words-250/2")

In [4]:
def get_max_length(df):
    """
    get max token counts from train data, 
    so we use this number as fixed length input to RNN cell
    """
    max_length = 0
    for row in df['review']:
        if len(row.split(" ")) > max_length:
            max_length = len(row.split(" "))
    return max_length

def get_word2vec_enc(reviews):
    """
    get word2vec value for each word in sentence.
    concatenate word in numpy array, so we can use it as RNN input
    """
    encoded_reviews = []
    for review in reviews:
        tokens = review.split(" ")
        review_encoding = []
        for i, token in enumerate(tokens):
            if i == 0:
                sentence_emb = embed([token])
            else:
                sentence_emb = np.concatenate((sentence_emb, embed([token])), axis=0)
        encoded_reviews.append(sentence_emb)
    return encoded_reviews
        
def get_padded_encoded_reviews(encoded_reviews):
    """
    for short sentences, we prepend zero padding so all input to RNN has same length
    """
    padded_reviews_encoding = []
    for enc_review in encoded_reviews:
        zero_padding_cnt = max_length - enc_review.shape[0]
        pad = np.zeros((1, 250))
        for i in range(zero_padding_cnt):
            enc_review = np.concatenate((pad, enc_review), axis=0)
        padded_reviews_encoding.append(enc_review)
    return padded_reviews_encoding

def sentiment_encode(sentiment):
    """
    return one hot encoding for Y value
    """
    if sentiment == 'positive':
        return [1,0]
    else:
        return [0,1]
    
def preprocess(df):
    """
    encode text value to numeric value
    """
    # encode words into word2vec
    reviews = df['review'].tolist()
    
    encoded_reviews = get_word2vec_enc(reviews)
    padded_encoded_reviews = get_padded_encoded_reviews(encoded_reviews)
    # encoded sentiment
    sentiments = df['sentiment'].tolist()
    encoded_sentiment = [sentiment_encode(sentiment) for sentiment in sentiments]
    X = np.array(padded_encoded_reviews)
    Y = np.array(encoded_sentiment)
    return X, Y 

# Preprocess (encode text to number)

In [5]:
movie_reviews_train = [
         {'review': 'this is the best movie', 'sentiment': 'positive'},
         {'review': 'i recommend you watch this movie', 'sentiment': 'positive'},
         {'review': 'it was waste of money and time', 'sentiment': 'negative'},
         {'review': 'the worst movie ever', 'sentiment': 'negative'}
    ]
df = pd.DataFrame(movie_reviews_train)

# max_length is used for max sequence of input
max_length = get_max_length(df)

train_X, train_Y = preprocess(df)

# Build Model

In [6]:
# LSTM model
model = Sequential()
model.add(LSTM(32))
model.add(Dense(2, activation='softmax'))

In [7]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train

In [8]:
print('Train...')
model.fit(train_X, train_Y,epochs=50)

Train...
Train on 4 samples
Epoch 1/50
4/4 [==============================] - 3s 868ms/sample - loss: 0.7104 - accuracy: 0.5000
Epoch 2/50
4/4 [==============================] - 0s 6ms/sample - loss: 0.6964 - accuracy: 0.5000
Epoch 3/50
4/4 [==============================] - 0s 8ms/sample - loss: 0.6828 - accuracy: 0.5000
Epoch 4/50
4/4 [==============================] - 0s 7ms/sample - loss: 0.6694 - accuracy: 0.7500
Epoch 5/50
4/4 [==============================] - 0s 7ms/sample - loss: 0.6563 - accuracy: 0.7500
Epoch 6/50
4/4 [==============================] - 0s 7ms/sample - loss: 0.6432 - accuracy: 1.0000
Epoch 7/50
4/4 [==============================] - 0s 7ms/sample - loss: 0.6303 - accuracy: 1.0000
Epoch 8/50
4/4 [==============================] - 0s 9ms/sample - loss: 0.6173 - accuracy: 1.0000
Epoch 9/50
4/4 [==============================] - 0s 9ms/sample - loss: 0.6042 - accuracy: 1.0000
Epoch 10/50
4/4 [==============================] - 0s 10ms/sample - loss: 0.5910 - accur

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  multiple                  36224     
_________________________________________________________________
dense (Dense)                multiple                  66        
Total params: 36,290
Trainable params: 36,290
Non-trainable params: 0
_________________________________________________________________


# Test
your model can predict correctly even for unseen words from training.  
This is the most benefit of using pretrained word embedding.  
Why? pretrained embedding will encode [better], [nice] to similar vector of [best]  
even if these words were not in train.  
therefore, the input vector to RNN is similar, so correct answers for even these unseen words.

In [10]:
"""
movie_reviews_train = [
         {'review': 'this is the best movie', 'sentiment': 'positive'},
         {'review': 'i recommend you watch this movie', 'sentiment': 'positive'},
         {'review': 'it was waste of money and time', 'sentiment': 'negative'},
         {'review': 'the worst movie ever', 'sentiment': 'negative'}
    ]
"""
movie_reviews_test = [
         {'review': 'it is better movie', 'sentiment': 'positive'},
         {'review': 'i suggest you see this movie', 'sentiment': 'positive'},
         {'review': 'it was just throwing 20 dollars away', 'sentiment': 'negative'},
         {'review': 'worse than any show', 'sentiment': 'negative'},
         {'review': 'nice movie, so love it', 'sentiment': 'positive'},
         {'review': 'terrible', 'sentiment': 'negative'}
    ]
test_df = pd.DataFrame(movie_reviews_test)

test_X, test_Y = preprocess(test_df)

score, acc = model.evaluate(test_X, test_Y, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

6/1 - 1s - loss: 0.1969 - accuracy: 1.0000
Test score: 0.19689838588237762
Test accuracy: 1.0
